# Loading

In [6]:
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import openai
import string
import json
import copy
import re
import time

import os
from dotenv import load_dotenv


In [83]:
load_dotenv()
model="gemini-1.5-flash"

gemini_api_key = os.getenv("GEMINI_API_KEY_1_5_THREE")

client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

try:
    resp = client.chat.completions.create(
        model="gemini-1.5-flash",
        messages=[{"role": "user", "content": "Ping"}]
    )
    print("Success:", resp.choices[0].message.content)
except Exception as e:
    # <-- note the closing ')' here
    print("Request failed:", type(e).__name__, e)

Success: Pong!



# Work Arrangement

In [8]:
train_df = pd.read_csv('../data/work_arrangements_development_set.csv')
test_df = pd.read_csv('../data/work_arrangements_test_set.csv')

In [9]:
train_df.iloc[48]

id                                                 80058389
job_ad    Job title: Tax Agent Training and Mentor Manag...
y_true                                               Hybrid
Name: 48, dtype: object

# RBIC Functions

### RB


In [10]:
#== step 1: role explanation
def step_1():
    messages = [
        {"role": "system", "content": "You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite."},
        {"role": "user", "content": "Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?"}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

In [11]:
#== few-shot examples
def fewshot(messages):
    # messages.append({
    #     "role": "user",
    #     "content": "I will provide you with some examples on how to accomplish your task"
    # })

    # response = client.chat.completions.create(
    #     model=model,
    #     messages=messages
    # )
    # messages.append({"role": "assistant", "content": response.choices[0].message.content})

    few_shot_indices = [20,23,36,48]

    for i in few_shot_indices:
        desc = {
          "job_ad": train_df.iloc[i].job_ad
        }
        desc_str = str(desc)

        # add the description
        messages.append({
          "role": "user",
          "content": desc_str
        })

        label = train_df.iloc[i].y_true
        label_str = str(label)

        # add the output
        messages.append({
            "role": "assistant",
            "content": label_str
        })

    return messages

In [12]:
#== step 2: setting sub-task --> ask for seniority patterns
def step_2(messages):
    messages.append({
        "role": "user",
        "content": "As a work arrangements classifier, what are some common phrases or patterns that indicate a work arrangements in a job description?"
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    return response.choices[0].message.content, messages

### IC

In [13]:
#== step 3: presence of seniority (skipped)
def step_3(messages_static, desc_str):
    messages = copy.deepcopy(messages_static)

    messages.append({
        "role": "user",
        "content": f"{desc_str} does this job description include any work-arrangements related information? Just respond with 'Yes' or 'No'."
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})

    p3_content = response.choices[0].message.content

    # clean and check the response
    response_p3 = p3_content.translate(str.maketrans('', '', string.punctuation))
    response_p3 = response_p3.strip().lower()[:3]
    #print(f"step3 messages: {len(messages)}")
    return response_p3, messages

In [14]:
#== step 4: iterative coaching/finding clues to prevent hallucination
def step_4(response_p3, messages):
    messages.append({
            "role": "user",
            "content": "Extract the work arrangements related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
        })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    #print(f"step4 messages: {len(messages)}")

    return response.choices[0].message.content, messages

In [56]:
#== step 5: use the clue to generate the final output
def step_5(messages):
    messages.append({
        "role": "user",
        "content": (
            "Based on the extracted work-arrangement clue and the text verbatim, return a structured work arrangements classification in the format {\"work_arrangements_label\": \"work arrangements classification\"}. Return only the json object with no extra details."
                        
        )
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    answer_str = response.choices[0].message.content

    print(messages)

    return answer_str

In [57]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
    response, messages = step_1()
    if verbose: print(f"RB step 1: {response}\n")

    response, messages = step_2(messages)
    if verbose: print(f"RB step 2: {response}\n")

    if add_fewshot:
        messages = fewshot(messages)
        if verbose: print(f"Fewshot examples added\n")
    return messages

In [58]:
def RBIC(messages, desc_str, verbose=False):
    response_p3, messages_local = step_3(messages, desc_str)
    if verbose: print(f"IC step 1: {response_p3}\n")

    response, messages_local = step_4(response_p3, messages_local)
    if verbose: print(f"IC step 2: {response}\n")

    answer_str = step_5(messages_local)

    match = re.search(r'\{.*\}', answer_str)
    
    json_str = match.group(0)

    data = json.loads(json_str)
    

    # extract the value
    label = data["work_arrangements_label"]
    
    if verbose: print(f"IC step 3 (Final): {label}\n")
    if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

    return str(label)

# Testing

### Qualitative Tests

In [59]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.

The work arrangement labels are:

* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.

* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.

* **Onsite:** The employee is required to work primarily or exclusively from a company office or other designated work location.  There might be occasional exceptions for things like business travel, but the core expectation is daily presence in the office.


RB step 2: Identifying work arrangements requires carefully examining the job description for keywords and contextual clues. Here are some common p

In [69]:
ind = 2

#[2081,1,5,12,3]
''' desc = {
          "job_ad": train_df.iloc[i].job_ad
        }'''


desc = {
    "job_ad": test_df.iloc[ind].job_ad
  }
desc_str = str(desc)

label = test_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 5
IC step 1: yes

IC step 2: ```json
{"Clue": ["Permanent work from home", "Complete WFH set-up", "Flexible working environment"]}
```


[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': "Work arrangement refers to where and how an employee performs their job.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels I use are:\n\n* **Remote:** The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.  This usually implies significant flexibility in work hours, subject to meeting deadlines and project requirements.\n\n* 

In [21]:
test_df.iloc[ind].y_true

'Remote'

### Quantitative Tests

In [22]:
print(test_df.iloc[21])

id                                                 79542140
job_ad    Job title: Marketing & Communications Coordina...
y_true                                               Hybrid
Name: 21, dtype: object


In [60]:
test_pred_df2 = pd.DataFrame(columns=["y_pred"])
test_pred_df2


,y_pred


In [25]:
print(len(test_df))

99


In [35]:
test_df[['y_true']].to_csv('work_arrangements_set_ans.csv', index=False)

Testing 

In [61]:
for i in tqdm(range(len(test_df))):
    desc = {"job_ad": test_df.iloc[i].job_ad}
    desc_str = str(desc)

    # keep retrying this same i until it succeeds
    while True:
        try:
            label_pred = RBIC(messages_static, desc_str)
            break  # exit retry loop on success
        except:
            print(f"Rate limited on row {i}, sleeping 60 seconds before retry…")
            time.sleep(60)

    # now record the successful prediction
    test_pred_df2.loc[len(test_pred_df2)] = label_pred


  1%|          | 1/99 [00:02<03:53,  2.39s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels are:\n\n* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.\n\n* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.\n\n* **Onsite:** The employee is requir

  2%|▏         | 2/99 [00:04<03:38,  2.25s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels are:\n\n* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.\n\n* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.\n\n* **Onsite:** The employee is requir

  3%|▎         | 3/99 [00:06<03:40,  2.30s/it]

...

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels are:\n\n* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.\n\n* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.\n\n* **Onsite:** The employee is requir

 99%|█████████▉| 98/99 [22:32<00:17, 17.85s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels are:\n\n* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.\n\n* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.\n\n* **Onsite:** The employee is requir

100%|██████████| 99/99 [22:33<00:00, 13.67s/it]

[{'role': 'system', 'content': 'You are an expert job-ad annotator. Your job is to read a job description and classify its work arrangement into exactly one of: Remote, Hybrid, or OnSite.'}, {'role': 'user', 'content': 'Based on your role, can you briefly explain what work arrangement means, and what work arrangement labels look like?'}, {'role': 'assistant', 'content': 'Work arrangement refers to where and how an employee performs their job duties.  It describes the physical location(s) and schedule flexibility involved.\n\nThe work arrangement labels are:\n\n* **Remote:**  The employee works exclusively from a location of their choosing, typically their home, and does not regularly work from a company office.\n\n* **Hybrid:** The employee splits their work time between working remotely and working from a company office or other designated work location.  This often involves a set schedule or a flexible arrangement agreed upon with the employer.\n\n* **Onsite:** The employee is requir

In [75]:
test_pred_df2.to_csv('work_arrangements_labelled_test_set_gemini_preds_few_shot.csv', index=False)

In [3]:
import pandas as pd
from sklearn.metrics import classification_report, f1_score, accuracy_score  # ← add accuracy_score

# 1. Load your files
preds = pd.read_csv('work_arrangements_labelled_test_set_gemini_preds_few_shot.csv')
true  = pd.read_csv('work_arrangements_set_ans.csv')

# 2. Extract the columns
y_pred = preds['y_pred']
y_true = true['y_true']

# 3. Define your label set
labels = ['OnSite', 'Hybrid', 'Remote']

# 4. Print the detailed classification report
print(classification_report(
    y_true,
    y_pred,
    labels=labels,
    target_names=labels,
    digits=3
))

# 5. Compute and print key F₁ metrics
print("Macro F₁:   ", f1_score(y_true, y_pred, average='macro',    labels=labels))
print("Weighted F₁:", f1_score(y_true, y_pred, average='weighted', labels=labels))
print("Micro F₁:   ", f1_score(y_true, y_pred, average='micro',    labels=labels))

# ← NEW: compute accuracy
print("Accuracy:   ", accuracy_score(y_true, y_pred))




              precision    recall  f1-score   support

      OnSite      0.976     0.870     0.920        46
      Hybrid      0.857     0.889     0.873        27
      Remote      0.929     1.000     0.963        26

   micro avg      0.928     0.909     0.918        99
   macro avg      0.920     0.919     0.918        99
weighted avg      0.931     0.909     0.918        99

Macro F₁:    0.9184101551917644
Weighted F₁: 0.9181770098523843
Micro F₁:    0.9183673469387755
Accuracy:    0.9090909090909091


# Regular prompting

In [21]:
# df to store model predictions
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [77]:
messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to classify a job ad into one of the work arrangements label - (OnSite, Hybrid, Remote) from job descriptions."},
]

In [78]:
test_pred_df3 = pd.DataFrame(columns=["y_pred"])
test_pred_df3

,y_pred


In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {"job_ad": test_df.iloc[i].job_ad}
    desc_str = str(desc)

    # rebuild messages for this row
    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract work-arrangements information from the job ad. "
            "Return a structured work-arrangements label in the format "
            "{\"work_arrangements_label\": \"work arrangements classification\"}. "
            "Return only the JSON object with no extra details."
        )
    })

    # retry the API call until it succeeds, waiting 60 s on error
    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages
            )
            break
        except:
            print(f"Error on row {i} sleeping 60 s before retry…")
            time.sleep(60)

    # extract the model’s answer text
    answer_str = response.choices[0].message.content

    # parse out the JSON
    try:
        match = re.search(r'\{.*\}', answer_str)
        json_str = match.group(0)
        data = json.loads(json_str)
        label = data["work_arrangements_label"]
    except (AttributeError, json.JSONDecodeError):
        print(f"Failed to parse JSON on row {i}: {answer_str}")
        label = "ERROR " + answer_str

    # append to the next empty row
    test_pred_df3.loc[len(test_pred_df3)] = label

# finally, save your results
test_pred_df3.to_csv(
    'work_arrangements_labelled_test_set_gemini_rbic_preds_simple_prompt.csv',
    index=False
)

 16%|█▌        | 16/99 [00:10<00:47,  1.74it/s]

Error on row 16 sleeping 60 s before retry…


 31%|███▏      | 31/99 [01:22<00:48,  1.41it/s]

Error on row 31 sleeping 60 s before retry…


 48%|████▊     | 48/99 [02:36<00:32,  1.59it/s]

...

 65%|██████▍   | 64/99 [03:49<00:23,  1.48it/s]

Error on row 64 sleeping 60 s before retry…


 90%|████████▉ | 89/99 [05:08<00:05,  1.82it/s]

Error on row 89 sleeping 60 s before retry…


100%|██████████| 99/99 [06:17<00:00,  3.82s/it]


In [54]:
test_pred_df3

,y_pred
0,OnSite
1,OnSite
2,Remote
3,Hybrid
4,OnSite
...,...
94,Hybrid
95,Hybrid
96,OnSite
97,OnSite


In [80]:

from sklearn.metrics import classification_report, f1_score

# 1. Load your files
preds = pd.read_csv('work_arrangements_labelled_test_set_gemini_rbic_preds_simple_prompt.csv')
true  = pd.read_csv('work_arrangements_set_ans.csv')

# 2. Extract the columns
y_pred = preds['y_pred']
y_true = true['y_true']

# 3. Define your label set
labels = ['OnSite', 'Hybrid', 'Remote']

# 4. Print the detailed classification report
print(classification_report(
    y_true,
    y_pred,
    labels=labels,
    target_names=labels,
    digits=3
))

# 5. Compute and print key F₁ metrics
print("Macro F₁:   ", f1_score(y_true, y_pred, average='macro', labels=labels))
print("Weighted F₁:", f1_score(y_true, y_pred, average='weighted', labels=labels))
print("Micro F₁:   ", f1_score(y_true, y_pred, average='micro', labels=labels))



              precision    recall  f1-score   support

      OnSite      0.955     0.913     0.933        46
      Hybrid      0.857     0.889     0.873        27
      Remote      0.926     0.962     0.943        26

    accuracy                          0.919        99
   macro avg      0.913     0.921     0.916        99
weighted avg      0.920     0.919     0.919        99

Macro F₁:    0.9164856108252334
Weighted F₁: 0.9194471877147692
Micro F₁:    0.9191919191919192
